# 3 Qubit bit flip code
We have seen in the introduction that we can have multiple types of errors. One of them is when you "store qubits" for a long period of time. We can correct a storage error but only when the error is an X gate (there are other methods for fixing multiple gates, but for now let's just focus on correcting an X gate). To solve this problem we need two additional qubits for each qubit that we want to correct. The main idea behind this and every other error correction code is that we'll use the additional qubits to check for errrors. In this way we avoid collapsing the superposition of our original qubit.

In this post, we'll present you an execution of the bit flip code by encoding a string into qubits and then meassuring. We do this in in order to see the effects of the noise and how destructive they can be for some small and inocent calculations.

In this code we make the assumption that the error happens in one part of the circuit. And also, the error only affects one qubit at a time.

First, let's define some function that will be useful for later.  The first two functions transform a string to bits and vice versa. The get_noise applies a x gate on a random qubit if a probability check up is met and finally codificate which codificate a bit string into it's equivalent in qubits. We simulate the noise in this way because it is easy to debug and check if we did everything correctly.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, ClassicalRegister
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import lookuptable_decoding
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.ignis.mitigation.measurement import (complete_meas_cal,CompleteMeasFitter)
from qiskit.visualization import plot_histogram
import random
backend = Aer.get_backend('qasm_simulator')

def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return ''.join([str(x) for x in result])

def frombits(bits):
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def get_noise(circuit,probability,qubits):
    random_number = random.uniform(0, 1)
    if(random_number <= probability):
        qubit = random.randint(0,len(qubits)-1)
        circuit.x(qubit)
    return circuit

def codificate(bitString):
    qubits = list()
    for i in range(len(bitString)):
        mycircuit = QuantumCircuit(1,1)
        if(bitString[i] == "1"):
            mycircuit.x(0)
        qubits.append(mycircuit)
    return qubits

Let's see the result for the string without any noise on the circuit

In [22]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

I like dogs


As you can see the string didn't suffer any transformation because we applied the inverse of the gates we used in the previous step. However, let's add some noise to check how the result will be affected.

In [25]:
m0 = tobits("I like dogs") 
qubits = codificate(m0)
measurements = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit = get_noise(qubit,0.2,range(qubit.num_qubits))
    qubit.measure(0,0)    
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))
print(frombits(measurements))

±npjg`f×2


And the string doesn't appear to say "I like dogs" anymore, even thought we use only a probability of 0.2 for an error to occur.

To address this issue we can use the 3 qubits bit flip code, which is describe by the following circuit:

In [6]:
cb = QuantumRegister(1,'code_qubit')
lq = QuantumRegister(4,'ancilla_qubit')
sb = ClassicalRegister(2,'syndrome_bit')
out = ClassicalRegister(1,'output_bit')
mycircuit = QuantumCircuit(cb,lq,sb,out)
mycircuit.cx(0,1)
mycircuit.cx(1,2)
mycircuit.barrier()
mycircuit = get_noise(mycircuit,0.2,range(3))
mycircuit.barrier()
mycircuit.cx(0,3)
mycircuit.cx(1,3)
mycircuit.cx(0,4)
mycircuit.cx(2,4)
mycircuit.measure(3,0)
mycircuit.measure(4,1)
mycircuit.draw()

░       ░                           
   code_qubit_0: ──■────────░───────░───■─────────■─────────────
                 ┌─┴─┐      ░ ┌───┐ ░   │         │             
ancilla_qubit_0: ┤ X ├──■───░─┤ X ├─░───┼────■────┼─────────────
                 └───┘┌─┴─┐ ░ └───┘ ░   │    │    │             
ancilla_qubit_1: ─────┤ X ├─░───────░───┼────┼────┼────■────────
                      └───┘ ░       ░ ┌─┴─┐┌─┴─┐  │    │  ┌─┐   
ancilla_qubit_2: ───────────░───────░─┤ X ├┤ X ├──┼────┼──┤M├───
                            ░       ░ └───┘└───┘┌─┴─┐┌─┴─┐└╥┘┌─┐
ancilla_qubit_3: ───────────░───────░───────────┤ X ├┤ X ├─╫─┤M├
                            ░       ░           └───┘└───┘ ║ └╥┘
 syndrome_bit: 2/══════════════════════════════════════════╩══╩═
                                                           0  1 
                                                                
   output_bit: 1/═══════════════════════════════════════════════

As we said in the introduction if we have those additional qubits we can check if an error occured during our execution. To identify this errors, we just have to check the following table to see what type of error we had (if any).

| Meassurement | Error location | Action |
|--------------|----------------|--------|
| 00           | No error       | No     |
| 01           | Ancilla bit    | No     |
| 10           | Ancilla bit    | No     |
| 11           | Code qubit     | X gate |

As you can see the only thing we need to do is to check the values of the meassurements and then apply a X gate.

Continuing with the proccess for the string "I like dogs", we have to to apply the error correction code.

In [74]:
for i in range(len(qubits)):
    cb = QuantumRegister(1,'code_qubit')
    lq = QuantumRegister(4,'ancilla_qubit')
    sb = ClassicalRegister(2,'syndrome_bit')
    out = ClassicalRegister(1,'output_bit')
    mycircuit = QuantumCircuit(cb,lq,sb,out)
    if(m0[i] == "1"):
        mycircuit.x(0)
    mycircuit.cx(0,1)
    mycircuit.cx(1,2)
    mycircuit = get_noise(mycircuit,0.2,range(3))
    mycircuit.cx(0,3)
    mycircuit.cx(1,3)
    mycircuit.cx(0,4)
    mycircuit.cx(2,4)
    mycircuit.measure(3,0)
    mycircuit.measure(4,1)
    qubits[i] = mycircuit

Finally, we check with the help of the four qubits if something had changed and then we apply a correction.

In [75]:
measurements = list()
raw_bits = list()
for i in range(len(qubits)):
    qubit = qubits[i]
    qubit.measure(0,2)
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    raw_bits.append(int(bits[0]))
for i in range(len(qubits)):
    qubit = qubits[i]
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    if(bits[2] == '1' and bits[3] == '0'):
        qubit.x(2)
    if(bits[2] == '0' and bits[3] == '1'):
        qubit.x(1)
    if(bits[2] == '1' and bits[3] == '1'):
        qubit.x(0)
    qubit.measure(0,2)
    result = execute(qubit, backend, shots=1, memory=True).result()
    bits = result.get_memory()[0]
    measurements.append(int(bits[0]))
print("without error correction the string was: " + frombits(raw_bits))
print("with error correction the string was: " + frombits(measurements))

without error correction the string was: I likg docS
with error correction the string was: I like dogs


As you can see the error correction works perfectly. But, in the real life we won't have just X-gates. We'll have a lot of other gates applied to our qubits. So we must find another way to correct errors.

## References:
Devitt, Simon & Munro, William & Nemoto, Kae. (2013). Quantum Error Correction for Beginners. Reports on progress in physics. Physical Society (Great Britain). 76. 076001. 10.1088/0034-4885/76/7/076001. 